In [6]:
#import ee
import numpy as np
import geopandas as gpd
import pandas as pd
import shapely
import glob
import os

#from ee_download import download_image_by_asset_path, Curler

In [5]:
ee.Authenticate()

Enter verification code:  4/1AY0e-g7lFofh-mCdiNgLjC--GPom_FP_MpNecxQp8cylWgb5ZVAQUJt5AsM



Successfully saved authorization token.


In [7]:
bbox = [-180, 45, 0, 75]
#2nd 1/2 world
bbox2 = [0,45,180,75]

# Failed attempt at using floats
#box_longs = np.arange(bbox[0],bbox[2], 1, dtype=np.int8).tolist()
#box_lats = np.arange(bbox[1],bbox[3], 1, dtype=np.int8).tolist()
box_longs = range(bbox[0], bbox[2], 1)
box_lats = range(bbox[1],bbox[3], 1)

## Original

Split into 2 tasks, while waiting for new global asset to be available.

In [18]:
all_boxes = []
for i in box_longs:
    for j in box_lats: 
        all_boxes.append([i,j,i+1,j+1])
print(len(all_boxes))
#all_boxes = all_boxes[2832:]
#print(len(all_boxes))

5400


In [19]:
bbox2 = [0,45,180,75]
box_longs = range(bbox2[0], bbox2[2], 1)
box_lats = range(bbox2[1],bbox2[3], 1)
for i in box_longs:
    for j in box_lats: 
        all_boxes.append([i,j,i+1,j+1])
print(len(all_boxes))

10800


In [ ]:
# This will download all the tiles we need each to a zip, 
# the files inside are all named the same so next step is to overwrite that with the zip name
for i in range(2832,len(all_boxes)):
    gee_geom = ee.Geometry.Rectangle(all_boxes[i])
    #print(gee_geom)
    
    downloaded_images = download_image_by_asset_path(
        asset_path = "users/alex-devseed/TTE_DATA_v201902v2-all",
        output_folder = "/projects/tmp/gee",
        region = gee_geom,
        idx = str(i)
    )


## Renaming files to be sane

In [64]:
bbox = [-180, 45, 0, 75]
box_longs = range(bbox[0], bbox[2], 1)
box_lats = range(bbox[1],bbox[3], 1)
all_boxes2 = []
for i in box_longs:
    for j in box_lats: 
        if i <= 0: 
            iname = f'W{abs(i)}'
        else:
            iname = f'E{(i)}'
        all_boxes2.append((f'{iname}N{j}',f'{i},{j},{i+1},{j+1}'))
print(len(all_boxes2))

5400


In [65]:
bbox2 = [0,45,180,75]
box_longs = range(bbox2[0], bbox2[2], 1)
box_lats = range(bbox2[1],bbox2[3], 1)
for i in box_longs:
    for j in box_lats: 
        if i <= 0: 
            iname = f'W{abs(i)}'
        else:
            iname = f'E{(i)}'
        all_boxes2.append((f'{iname}N{j}',f'{i},{j},{i+1},{j+1}'))
print(len(all_boxes2))

10800


In [71]:
def make_int_box(box):
    return [int(num) for num in box.split(',')]

In [72]:
box_table = pd.DataFrame(all_boxes2, columns= ["Name", "box"])
box_table["id"] = np.arange(box_table.shape[0])
polys = [shapely.geometry.box(*make_int_box(box)) for box in box_table["box"]]
box_table['gboxes'] = gpd.GeoSeries(polys)

In [73]:
geo_boxes = gpd.GeoDataFrame(box_table, geometry='gboxes')

In [74]:
tmp = '/projects/tmp/gee/'
geo_boxes.head()

,Name,box,id,gboxes
0,W180N45,"-180,45,-179,46",0,"POLYGON ((-179.00000 45.00000, -179.00000 46.0..."
1,W180N46,"-180,46,-179,47",1,"POLYGON ((-179.00000 46.00000, -179.00000 47.0..."
2,W180N47,"-180,47,-179,48",2,"POLYGON ((-179.00000 47.00000, -179.00000 48.0..."
3,W180N48,"-180,48,-179,49",3,"POLYGON ((-179.00000 48.00000, -179.00000 49.0..."
4,W180N49,"-180,49,-179,50",4,"POLYGON ((-179.00000 49.00000, -179.00000 50.0..."


In [75]:
def find_file(token, files):
    result = [match for match in files if token in match]
    if len(result) >= 1:
        return result[0]
    else:
        return None
    
def local_to_s3(url):
    return url.replace("/projects/maap-users", "s3://maap-ops-dataset/maap-users")

In [76]:
# make a list of zips
#glob find the tifs
zip_dir = "/projects/tmp/gee/"

local_zip = glob.glob(os.path.join(zip_dir,"*.zip"), recursive=False)
#tiles = [local_to_s3(tile) for tile in local_tiles]
# make a list of tifs
tif_dir = "/projects/maap-users/alexdevseed/gee"
local_tiles = glob.glob(os.path.join(tif_dir,"*.zip.tif"), recursive=False)
#tiles = [local_to_s3(tile) for tile in local_tiles]

# find matches for the rowid of the table and add columns
# calculate the s3 path and add column

In [77]:
geo_boxes['zip'] = [find_file(f"-{idx}.zip", local_zip) for idx in geo_boxes['id']]

In [78]:
geo_boxes['tif'] = [find_file(f"-{idx}.zip.tif", local_tiles) for idx in geo_boxes['id']]

In [79]:
geo_boxes['s3'] = local_to_s3(geo_boxes["tif"])

In [80]:
geo_boxes.to_file("/projects/maap-users/alexdevseed/gee/tile_index.geojson", driver="GeoJSON")
#geo_boxes.iloc[1]

## Debugging

In [3]:
graticule = gpd.read_file('grat2-5.geojson')
graticule.shape

(1728, 6)

In [5]:
all_boxes[5350:]

[[-2, 55, -1, 56],
 [-2, 56, -1, 57],
 [-2, 57, -1, 58],
 [-2, 58, -1, 59],
 [-2, 59, -1, 60],
 [-2, 60, -1, 61],
 [-2, 61, -1, 62],
 [-2, 62, -1, 63],
 [-2, 63, -1, 64],
 [-2, 64, -1, 65],
 [-2, 65, -1, 66],
 [-2, 66, -1, 67],
 [-2, 67, -1, 68],
 [-2, 68, -1, 69],
 [-2, 69, -1, 70],
 [-2, 70, -1, 71],
 [-2, 71, -1, 72],
 [-2, 72, -1, 73],
 [-2, 73, -1, 74],
 [-2, 74, -1, 75],
 [-1, 45, 0, 46],
 [-1, 46, 0, 47],
 [-1, 47, 0, 48],
 [-1, 48, 0, 49],
 [-1, 49, 0, 50],
 [-1, 50, 0, 51],
 [-1, 51, 0, 52],
 [-1, 52, 0, 53],
 [-1, 53, 0, 54],
 [-1, 54, 0, 55],
 [-1, 55, 0, 56],
 [-1, 56, 0, 57],
 [-1, 57, 0, 58],
 [-1, 58, 0, 59],
 [-1, 59, 0, 60],
 [-1, 60, 0, 61],
 [-1, 61, 0, 62],
 [-1, 62, 0, 63],
 [-1, 63, 0, 64],
 [-1, 64, 0, 65],
 [-1, 65, 0, 66],
 [-1, 66, 0, 67],
 [-1, 67, 0, 68],
 [-1, 68, 0, 69],
 [-1, 69, 0, 70],
 [-1, 70, 0, 71],
 [-1, 71, 0, 72],
 [-1, 72, 0, 73],
 [-1, 73, 0, 74],
 [-1, 74, 0, 75]]

In [15]:
gee_geom = ee.Geometry.Rectangle([-55,40,-54,41], 'EPSG:4326', False)

In [16]:
gee_geom

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Rectangle",
    "arguments": {
      "coordinates": {
        "constantValue": [
          -55,
          40,
          -54,
          41
        ]
      },
      "crs": {
        "functionInvocationValue": {
          "functionName": "Projection",
          "arguments": {
            "crs": {
              "constantValue": "EPSG:4326"
            }
          }
        }
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [4]:
i = 500
gee_geom = ee.Geometry.Rectangle(all_boxes[i])
print(gee_geom)
    
downloaded_images = download_image_by_asset_path(
        asset_path = "users/alex-devseed/TTE_DATA_v201902v2-class",
        output_folder = "/projects/tmp/gee",
        region = gee_geom,
        idx = str(i)
)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -164,
              66
            ],
            [
              -164,
              65
            ],
            [
              -163,
              65
            ],
            [
              -163,
              66
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})


In [37]:
image = ee.Image("users/alex-devseed/TTE_DATA_v201902v2-class")
path = image.getDownloadUrl({
            #'scale': 30,
            #'crs': 'EPSG:3310',
            'region': gee_geom
        })

In [39]:
c = Curler()
success = c.download(url=path, path="/projects/tmp/gee/test.zip")

In [32]:
from zipfile import is_zipfile, ZipFile
download_path = '/projects/tmp/gee/TTE_DATA_v201902v2-class.zip'

In [ ]:

with open(download_path, 'r') as zf:
        z = ZipFile(zf)
        downloaded_items = [os.path.join(output_folder,item) for item in z.namelist() if not item.endswith("tfw") ]
        z.extractall(path=output_folder)
        z.close()
        del z

In [33]:
with ZipFile(download_path, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()

File Name                                             Modified             Size
TTE_DATA_v201902v2-class.tte_classification.tif 2021-06-11 10:08:42        89175


In [41]:
[1,2,3,4]

[1, 2, 3, 4]